In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Lambda, Embedding, Flatten
from keras.models import Model
from keras import backend as K
from keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
# Load the CSV dataset
df = pd.read_csv("/content/drive/MyDrive/Song-Lyrics-Generation/data/processed/dataset.csv", index_col=[0])


In [ ]:
df = df.sample(n=10000, random_state=42)

In [ ]:
# Extract lyrics from the dataset
lyrics = df['Lyric'].tolist()

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["Lyric"].astype(str).str.lower())
total_words = len(tokenizer.word_index) + 1
tokenized_sentences = tokenizer.texts_to_sequences(df["Lyric"].astype(str))

In [ ]:
from tqdm import tqdm

def generate_sequences(tokenized_sentences):
    for i in tqdm(tokenized_sentences, desc="Generating sequences"):
        for t in range(1, len(i)):
            n_gram_sequence = i[: t + 1]
            yield n_gram_sequence

In [ ]:
sequence_generator = generate_sequences(tokenized_sentences)

# Find the maximum sequence length.
max_sequence_len = max(len(seq) for seq in tqdm(sequence_generator, desc="Calculating max sequence length"))

# Create a new generator for sequences.
sequence_generator = generate_sequences(tokenized_sentences)

# Pad sequences in smaller batches to save memory.
batch_size = 1000
padded_sequences = []

for batch in tqdm(iter(lambda: list(sequence_generator)[:batch_size], []), desc="Padding sequences"):
    padded_sequences.extend(keras.preprocessing.sequence.pad_sequences(batch, maxlen=max_sequence_len, padding="pre"))

input_sequences = np.array(padded_sequences)

Calculating max sequence length: 0it [00:00, ?it/s]
Calculating max sequence length: 105569it [00:00, 1055630.60it/s]
Calculating max sequence length: 211133it [00:00, 1029861.58it/s]
Calculating max sequence length: 328508it [00:00, 1094663.37it/s]
Calculating max sequence length: 446317it [00:00, 1127313.40it/s]
Calculating max sequence length: 559400it [00:00, 1128556.90it/s]
Calculating max sequence length: 672317it [00:00, 1086885.08it/s]
Calculating max sequence length: 782743it [00:00, 1092391.54it/s]
Calculating max sequence length: 893005it [00:00, 1095570.79it/s]
Calculating max sequence length: 1009560it [00:00, 1117145.44it/s]
Calculating max sequence length: 1126881it [00:01, 1134283.64it/s]
Calculating max sequence length: 1240421it [00:01, 1106703.67it/s]
Calculating max sequence length: 1356501it [00:01, 1122791.79it/s]
Calculating max sequence length: 1473776it [00:01, 1137684.15it/s]
Calculating max sequence length: 1595357it [00:01, 1161015.86it/s]
Calculating max se

In [ ]:
# Pad sequences to a fixed length
maxlen = 1000  # Set a reasonable maximum length
padded_sequences = pad_sequences(sequences, padding='post', dtype='int32', truncating='post', maxlen=maxlen)


In [ ]:
max_length = 1000  # Set your desired maximum length
truncated_sequences = [sequence[:max_length] for sequence in padded_sequences]


In [ ]:
maxlen = max(len(seq) for seq in input_sequences)
maxlen

4489

In [ ]:
len(truncated_sequences)

In [ ]:
# Split the data into training and testing sets
X_train, X_test = train_test_split(input_sequences, test_size=0.2, random_state=42)

In [ ]:
# Define VAE parameters
latent_dim = 32
intermediate_dim = 256
epsilon_std = 1.0
embedding_dim = 200
num_latent_vars = 3

In [ ]:
# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim), mean=0., stddev=epsilon_std)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
# Encoder
inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=maxlen)(inputs)
flatten_layer = Flatten()(embedding_layer)
h = Dense(intermediate_dim, activation='relu')(flatten_layer)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

# Sample z using reparameterization trick
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

In [ ]:
# Decoder
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(maxlen, activation='sigmoid')

h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

In [ ]:
from keras.layers import Layer
from keras.losses import binary_crossentropy
from keras import backend as K

class CustomVariationalLayer(keras.layers.Layer):
    def vae_loss(self, x, x_decoded_mean, z_mean, z_log_var):
        xent_loss = maxlen * binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        z_mean = inputs[2]
        z_log_var = inputs[3]
        loss = self.vae_loss(x, x_decoded_mean, z_mean, z_log_var)
        self.add_loss(loss, inputs=inputs)
        return x

In [ ]:
from keras.optimizers import Adam

# Instantiate VAE model
y = CustomVariationalLayer()([inputs, x_decoded_mean, z_mean, z_log_var])
vae = Model(inputs, y)
# Assuming you are using MSE as the loss function
vae.compile(optimizer=Adam(clipvalue=1.0, learning_rate = 0.00001), loss=None)


In [ ]:
#Print the summary of your VAE model to see the layer names
vae.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 4489)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, 4489, 200)            8314600   ['input_2[0][0]']             
                                                                                                  
 flatten (Flatten)           (None, 897800)               0         ['embedding[0][0]']           
                                                                                                  
 dense (Dense)               (None, 256)                  2298370   ['flatten[0][0]']             
                                                          56                                

In [ ]:
from keras.callbacks import Callback

class DebugCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Get the values at the end of each epoch
        x_values = self.validation_data[0]
        x_decoded_mean_values = self.model.predict(x_values)
        z_mean_values, z_log_var_values = self.model.get_layer('custom_variational_layer_1').predict(x_values)

        # Print or log the values
        print("x:", x_values)
        print("x_decoded_mean:", x_decoded_mean_values)
        print("z_mean:", z_mean_values)
        print("z_log_var:", z_log_var_values)

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = vae.fit(X_train, epochs=10, batch_size=32, shuffle=True, validation_data=(X_test, None), callbacks=[early_stopping])

Epoch 1/10
25/25 [==============================] - 9s 296ms/step - loss: -5441053.0000 - val_loss: -4776094.0000
Epoch 2/10
25/25 [==============================] - 4s 170ms/step - loss: -6206898.5000 - val_loss: -6353220.5000
Epoch 3/10
25/25 [==============================] - 3s 135ms/step - loss: -6872452.0000 - val_loss: -6316979.5000
Epoch 4/10
25/25 [==============================] - 4s 154ms/step - loss: -7228686.0000 - val_loss: -6415738.5000
Epoch 5/10
25/25 [==============================] - 3s 114ms/step - loss: -8187629.0000 - val_loss: -6290669.5000
Epoch 6/10
25/25 [==============================] - 3s 105ms/step - loss: -9278574.0000 - val_loss: -5973985.5000
Epoch 7/10
25/25 [==============================] - 3s 139ms/step - loss: -9044991.0000 - val_loss: -7852519.0000
Epoch 8/10
25/25 [==============================] - 5s 206ms/step - loss: -10466345.0000 - val_loss: -8529614.0000
Epoch 9/10
25/25 [==============================] - 3s 141ms/step - loss: -10435073.000

In [139]:
import numpy as np

def generate_lyric_vae(vae, tokenizer, latent_dim=32, maxlen=4489):
    # Generate new lyrics
    new_lyric_vector = np.random.normal(size=(1, 4489))
    decoded_lyric = vae.predict(new_lyric_vector)

    # Inverse transform the decoded lyric back to text
    decoded_lyric_text = tokenizer.sequences_to_texts([decoded_lyric.argmax(axis=-1) + 1])[0]

    return decoded_lyric_text

# Example usage:
generated_lyric = generate_lyric_vae(vae, tokenizer)
print("Generated Lyric:\n", generated_lyric)


1/1 [==============================] - 0s 19ms/step
Generated Lyric:
 bringin


In [191]:
import numpy as np

def generate_lyric_vae_with_seed(vae, tokenizer, seed_text, latent_dim=32, maxlen=4489, n_words=50, temperature=1.0):
    # Tokenize the seed text
    seed_sequence = tokenizer.texts_to_sequences([seed_text])[0]
    print(seed_sequence)
    seed_padded = pad_sequences([seed_sequence], maxlen=maxlen, padding='pre', truncating='pre')

    # Generate new lyrics word by word
    generated_lyric = seed_text
    for _ in range(n_words):
        # Predict the next word based on the seed text
        predicted = vae.predict(seed_padded, verbose=0)[0]

        # Sample from the modified distribution using temperature
        predicted = np.log(predicted) / temperature
        exp_preds = np.exp(predicted)
        normalized_preds = exp_preds / np.sum(exp_preds)
        predicted_index = np.argmax(np.random.multinomial(1, normalized_preds, 1))

        # Convert the index back to a word
        output_word = tokenizer.index_word.get(predicted_index, "")

        # Update the seed text for the next iteration
        seed_text += " " + output_word
        seed_sequence = tokenizer.texts_to_sequences([seed_text])[0]
        seed_padded = pad_sequences([seed_sequence], maxlen=maxlen, padding='pre', truncating='pre')

        # Update the generated lyric
        generated_lyric += " " + output_word

    return generated_lyric

# Example usage:
seed_text = "I want to"
generated_lyric = generate_lyric_vae_with_seed(vae, tokenizer, seed_text, n_words= 5, temperature=10)
print("Generated Lyric:\n", generated_lyric)






[1, 63, 4]


<ipython-input-191-b485f023f7f9>:16: RuntimeWarning: divide by zero encountered in log
  predicted = np.log(predicted) / temperature


Generated Lyric:
 I want to cheers disrespect cheers disrespect floss
